In [1]:
# Youngwoo Kim
!hdfs dfs -D dfs.replication=1 -cp -f data/*.csv hdfs://nn:9000/

In [2]:
!hdfs dfs -du -h hdfs://nn:9000/*

328      328      hdfs://nn:9000/action_taken.csv
317      317      hdfs://nn:9000/agency.csv
521.0 K  521.0 K  hdfs://nn:9000/arid2017_to_lei_xref_csv.csv
311.6 K  311.6 K  hdfs://nn:9000/counties.csv
237      237      hdfs://nn:9000/denial_reason.csv
109      109      hdfs://nn:9000/edit_status.csv
180      180      hdfs://nn:9000/ethnicity.csv
166.8 M  166.8 M  hdfs://nn:9000/hdma-wi-2021.csv
41       41       hdfs://nn:9000/hoepa.csv
114      114      hdfs://nn:9000/lien_status.csv
65       65       hdfs://nn:9000/loan_purpose.csv
79       79       hdfs://nn:9000/loan_type.csv
129.6 K  129.6 K  hdfs://nn:9000/msamd.csv
122      122      hdfs://nn:9000/owner_occupancy.csv
92       92       hdfs://nn:9000/preapproval.csv
127      127      hdfs://nn:9000/property_type.csv
387      387      hdfs://nn:9000/purchaser_type.csv
252      252      hdfs://nn:9000/race.csv
144      144      hdfs://nn:9000/sex.csv
955      955      hdfs://nn:9000/states.csv
2.6 M    2.6 M    hdfs://nn:9000/trac

In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 21:51:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
banks_df = spark.read.options(header=True, inferSchema=True).csv("hdfs://nn:9000/arid2017_to_lei_xref_csv.csv")

In [5]:
type(banks_df)

pyspark.sql.dataframe.DataFrame

In [6]:
banks_df.rdd.take(3)

[Row(respondent_name='First National Bank', arid_2017='110004', lei_2018='5493003EW6T31TGECO83', lei_2019='5493003EW6T31TGECO83', lei_2020='5493003EW6T31TGECO83'),
 Row(respondent_name='First Mid Bank & Trust, National Association', arid_2017='110045', lei_2018='549300XOTES5TCS8T794', lei_2019='549300XOTES5TCS8T794', lei_2020='549300XOTES5TCS8T794'),
 Row(respondent_name='First Hope Bank, A National Banking Association', arid_2017='110118', lei_2018='5493003XLOX5FDT9R120', lei_2019='5493003XLOX5FDT9R120', lei_2020='5493003XLOX5FDT9R120')]

In [7]:
#q1
banks_df.rdd.filter(lambda x:"first" in x.respondent_name.lower()).count()

525

In [8]:
from pyspark.sql.functions import lower

In [9]:
#q2
banks_df.filter(lower(banks_df.respondent_name).like("%first%")).count()

525

In [25]:
#q3
banks_df.write.saveAsTable("banks", mode="overwrite")
spark.sql("""
            SELECT * FROM banks
            WHERE LOWER(respondent_name) LIKE '%first%'
            """).count()

525